Адаптируем построение бандла из задачи поиска ошибки согласования рода к задаче поиска антецедентов.

Посмотрим как работает CoreExtractor - сущность, которая по слову вернет все признаки. Умеет работать с любыми индексами.

In [65]:
from sklearn import datasets
import pandas as pd
from yo_fluq_ds import *
from tg.common.ml import batched_training as bt
from tg.common import DataBundle, Loc
from tg.common.ml.batched_training import IndexedDataBundle
from tg.grammar_ru.ml.components.core_extractor.extractor import CoreExtractor
from tg.grammar_ru.ml.components.plain_context_builder import PlainContextBuilder
from tg.grammar_ru.ml.components.contextual_binding import ContextualBinding, ContextualNetworkType
from tg.common.ml.batched_training import mirrors as btm
from tg.common.ml.batched_training import Batcher
from tg.grammar_ru.common import Separator

In [66]:
from tg.grammar_ru.common import Loc
from tg.grammar_ru.ml.features import PyMorphyFeaturizer

db = DataBundle.load(Loc.bundles_path/'antcd/wwd')
db.index

,pronoun_word_id,pronoun_sentence_id,candidate_word_id,candidate_sentence_id,candidate_distance,is_match,word_distance
sample_id,,,,,,,
0,7,0,4,0,1,1,3
1,7,0,5,0,0,0,2
2,514887,514880,514884,514880,1,1,3
3,514887,514880,514885,514880,0,0,2
4,1903357,1903337,1903342,1903335,3,0,12
...,...,...,...,...,...,...,...
101060,22837552,22837430,22837513,22837429,4,0,33
101061,22837552,22837430,22837524,22837430,3,0,25
101062,22837552,22837430,22837537,22837430,2,0,14


In [67]:
from tg.common.ml.batched_training import train_display_test_split

db.index = db.index.iloc[:1000]
db.index = db.index.rename(columns={'is_match': 'label'})
db.index['split'] = train_display_test_split(db.index)
db.index

,pronoun_word_id,pronoun_sentence_id,candidate_word_id,candidate_sentence_id,candidate_distance,label,word_distance,split
sample_id,,,,,,,,
0,7,0,4,0,1,1,3,test
1,7,0,5,0,0,0,2,train
2,514887,514880,514884,514880,1,1,3,test
3,514887,514880,514885,514880,0,0,2,test
4,1903357,1903337,1903342,1903335,3,0,12,train
...,...,...,...,...,...,...,...,...
995,2011233,2010938,2011213,2010935,2,0,14,train
996,2011233,2010938,2011223,2010937,1,0,7,test
997,2011233,2010938,2011227,2010937,0,1,4,train


In [68]:
idb = IndexedDataBundle(db.index, db)
idb

In [69]:
Separator.Viewer().tooltip("word_id").to_html_display(
    idb.bundle.src.loc[idb.bundle.src.sentence_id == 0])


CoreExtractor принимает список extractor'ов и применяет их к бандлу. Потом нужно подать его ContextualBinding. TODO да?

##### Пример CoreExtractor

In [70]:
core = CoreExtractor(join_column='pronoun_word_id')
core.fit(idb)


In [71]:
extracted = core.extract(idb)
extracted

,pymorphy_score,pymorphy_delta_score,pymorphy_alternatives,pymorphy_POS_NPRO,pymorphy_animacy_NULL,pymorphy_gender_masc,pymorphy_gender_femn,pymorphy_number_sing,pymorphy_case_accs,pymorphy_case_nomn,...,slovnet_morph_Voice_NULL,slovnet_morph_Degree_NULL,slovnet_morph_Foreign_NULL,slovnet_morph_Variant_NULL,slovnet_morph_Polarity_NULL,slovnet_syntax_relation_nmod,slovnet_syntax_relation_det,slovnet_syntax_relation_obj,slovnet_syntax_relation_nsubj,slovnet_syntax_relation_iobj
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.039619,0.053981,0.247520,1.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,-0.908753,-0.877471,0.097863,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
996,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
997,1.164046,0.228254,-3.433975,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


* Для каждого слова получили все признаки из pymorphy, slovnet...

* Нам необходим контекст слова

Будем использовать `PlainContextBuilder`.  Он построит двойной индекс.

* хотим исключить само слово из контекста, поэтому `include_zero_offset=False`

#### Пример PlainContextBuilder

In [72]:
pronoun_pcb = PlainContextBuilder(include_zero_offset=True,
                                  left_to_right_contexts_proportion=0.5)
pronoun_pcb.sentence_id_column_name = 'pronoun_sentence_id'
pronoun_pcb.word_id_column_name = 'pronoun_word_id'
pronoun_contexts = pronoun_pcb.build_context(idb, context_size=5)
pronoun_contexts  # TODO у некоторых только offsset == 1


another_word_id
sample_id offset                 
0         0                     7
1         0                     7
2         0                514887
3         0                514887
4         0               1903357
...                           ...
997       2               2011235
998       1               2021284
          2               2021285
999       1               2021284
          2               2021285

[4027 rows x 1 columns]

In [73]:
cdf = pronoun_contexts.reset_index()
cdf.loc[cdf.sample_id == 2]

,sample_id,offset,another_word_id
2,2,0,514887
1004,2,-1,514886
1005,2,-2,514885
2031,2,1,514888
2032,2,2,514889


#### WordContextAssemblyPoint (~ ContextualBinding)

Задача AssemblyPoint'a (он же ContextualBinding) - порождать экстракторами батчи в том виде, в котором они будут приняты сетью. Для LSTM будет 3d-тензор. Для Plain-сети будет плоский датафрейм.

In [74]:
def get_plain_context(pcb):
    return ContextualBinding(
    name='plain_context',
    context_length=3,
    network_type=ContextualNetworkType.Plain,
    hidden_size=[30],
    context_builder=pcb,
    extractor=CoreExtractor(join_column='another_word_id'),
    debug=False
)

pronoun_plain_context = get_plain_context(pronoun_pcb)

**Пояснение:**
По умолчанию CoreExtractor пытается мерджить по word_id. В нашем случае CoreExtractor отработает после создания PlainContextBuilder'ом двойного индекса. 
Для добавления признаков слов из контекста будем join'ить по столбцу another_word_id. Поэтому join_column='another_word_id'

Для создания согласованных экстракторов и сетей у AssemblyPoint есть методы create_extractor и create_network.

У такой сети на последнем слое hidden_size нейронов. Ее выход можно подать в другую сеть, которая будет выдавать вероятности классов.

In [75]:
pronoun_context_extractor = pronoun_plain_context.create_extractor(task=None, bundle=idb)
pronoun_context_extractor.fit(idb)
# not_batch = core_extractor.extract(idb)


In [76]:
candidate_context_extractor = candidate_plain_context.create_extractor(task=None, bundle=idb)
candidate_context_extractor.fit(idb)

In [77]:
factory = pronoun_plain_context.create_network_factory(
    task=None, input=None)  # None это ок. это legacy


#### Batcher

Иногда нам нужны фичи контекстов нескольких слов. Например, в задаче поиска антецедентов это местоимение и два существительных - потенциальные антецеденты. Для каждого из этих слов создадим экстрактор (для каждого экстрактора будет отдельная голова нейросети). Batcher примет список этих экстракторов и создаст батч.

`+` нам понадобится экстрагировать лейблы

In [78]:
from tg.grammar_ru.ml.components.training_task_factory import Conventions
from tg.common.ml import dft
# экстрагирует лейблы. Получается one-hot df. Лейблы пойдут в loss function.


def get_binary_label_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=None))
    return label_extractor


def test_extractor(extractor, bundle):
    extractor.fit(bundle)
    return extractor.extract(bundle)

test_extractor(get_binary_label_extractor(), idb).head(10)


,label
sample_id,
0,1
1,0
2,1
3,0
4,0
5,0
6,1
7,0
8,0


In [79]:
batcher = Batcher(batch_size=1000, extractors=[
                  pronoun_context_extractor, get_binary_extractor()])
batch = batcher.fit_extract(idb)


In [80]:
network = factory.create_network(task=None, input=batch)
network(batch).shape

torch.Size([1000, 30])

+++++++++++++++++++++++
+++++++++++++++++++++++
+++++++++++++++++++++++

In [81]:
import torch
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions
from tg.common.ml.batched_training.torch.networks.extracting_network import FeedForwardNetwork
from tg.common.ml.batched_training.torch.networks.simple_networks import FullyConnectedNetwork, ParallelNetwork

def get_head_factory(sentence_id_column_name, word_id_column_name):
    pcb = PlainContextBuilder(include_zero_offset=True, left_to_right_contexts_proportion=0.5)
    pcb.sentence_id_column_name = sentence_id_column_name
    pcb.word_id_column_name = word_id_column_name
    plain_context = ContextualBinding(
        name='plain_context',
        context_length=3,
        network_type=ContextualNetworkType.Plain,
        hidden_size=[30],
        context_builder=pcb,
        extractor=CoreExtractor(join_column='another_word_id'),
        debug=False
    )
    return plain_context.create_network_factory(task=None, input=None)


class HeadsUnionNetwork(torch.nn.Module):
    def __init__(self,
                 sizes: List[int],
                 heads,
                 outs,
                 output: Union[None, torch.Tensor, int] = None):
        super(HeadsUnionNetwork, self).__init__()
        heads_forward_networks = dict()
        for i in range(len(heads)):
            fully_connected = FullyConnectedNetwork(sizes=[3], input=outs[i].shape[1], output=output)
            heads_forward_networks[str(i)] = FeedForwardNetwork(heads[i], fully_connected, torch.nn.Softmax(dim=1))
        parallel_network = ParallelNetwork()
        parallel_network.networks = torch.nn.ModuleDict(heads_forward_networks)

    def forward(self, input):
        heads_outputs = parallel_network(input)
        for output in heads_outputs.values():
            #aggregate results
            print(output)
    

class MultiheadNetworkFactory:
    def __init__(self, *head_factories):
        self.head_factories = [*head_factories]

    def create_network(self, task, input):  # input is batch ~ sample
        heads = [f.create_network(task=None, input=input) for f in self.head_factories]
        outs = [h(input) for h in heads]

        return HeadsUnionNetwork(sizes=[3], heads=heads, outs=outs, output=input['label'].shape[1])

pronoun_head_factory = get_head_factory('pronoun_sentence_id', 'pronoun_word_id')
candidate_head_factory = get_head_factory('candidate_sentence_id', 'candidate_word_id')
network_factory = MultiheadNetworkFactory(pronoun_head_factory, candidate_head_factory)
assembled_network = network_factory.create_network(task=None, input=batch)

In [89]:
class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add_sklearn(roc_auc_score)
        self.instantiate_default_task(epoch_count=10, batch_size=1000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(data, [pronoun_context_extractor, get_binary_label_extractor()])
        self.setup_model(network_factory)
        
task = ClassificationTask()
result = task.run(db)

KeyError: 'split'

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [83]:
mini_batch_indices = batcher.get_mini_batch_indices(mini_batch_size = 10, batch = batch)
mini_batch = batcher.get_mini_batch(index = mini_batch_indices[0], batch = batch)
mini_batch['label'].shape

(10, 1)

In [84]:
# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, input)
output.backward()
# # Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
# output = loss(input, target)
# output.backward()
output

tensor(0.8770, grad_fn=<DivBackward1>)

In [85]:
def train_loop(model, loss_fn, optimizer):
    for mini_batch_num, mini_batch_index in enumerate(mini_batch_indices):
        mini_batch = batcher.get_mini_batch(index = mini_batch_indices[mini_batch_num], batch = batch)
        # Compute prediction and loss
        pred = model(mini_batch)
        loss = loss_fn(pred, torch.tensor(mini_batch['label'].values, dtype=torch.float32))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{num_batches:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [86]:
learning_rate = 1
optimizer = torch.optim.SGD(assembled_network.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(assembled_network, loss, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
print("Done!")


ValueError: optimizer got an empty parameter list

In [ ]:
mini_batch['label']

In [ ]:
out = assembled_network(batch)
out.shape


In [ ]:
import torch


class ClassificationNetwork(torch.nn.Module):
    def __init__(self, hidden_size, sample):
        super(ClassificationNetwork, self).__init__()
        self.hidden = torch.nn.Linear(sample['features'].shape[1], hidden_size)
        self.output = torch.nn.Linear(hidden_size, sample['label'].shape[1])

    def forward(self, input):
        X = input['features']
        X = torch.tensor(X.astype(float).values).float()
        X = self.hidden(X)
        X = torch.sigmoid(X)
        X = self.output(X)
        X = torch.sigmoid(X)
        return X


In [ ]:
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions
from tg.common.ml import dft


class MulticlassMetrics(bt.Metric):
    def __init__(self, add_accuracy=True, add_rating=False):
        self.add_accuracy = add_accuracy
        self.add_rating = add_rating

    def get_names(self):
        result = []
        if self.add_accuracy:
            result.append('accuracy')
        if self.add_rating:
            result.append('rating')
        return result

    def measure(self, df, _):
        prefix = 'true_label_'
        labels = []
        for c in df.columns:
            if c.startswith(prefix):
                labels.append(c.replace(prefix, ''))

        def ustack(df, prefix, cols, name):
            df = df[[prefix+c for c in cols]]
            df.columns = [c for c in cols]
            df = df.unstack().to_frame(name)
            return df

        predicted = ustack(df, 'predicted_label_', labels, 'predicted')
        true = ustack(df, 'true_label_', labels, 'true')
        df = predicted.merge(true, left_index=True,
                             right_index=True).reset_index()
        df.columns = ['label', 'sample', 'predicted', 'true']
        df = df.feed(fluq.add_ordering_column(
            'sample', ('predicted', False), 'predicted_rating'))

        match = (df.loc[df.predicted_rating ==
                 0].set_index('sample').true > 0.5)
        rating = df.loc[df.true > 0.5].set_index('sample').predicted_rating
        result = []
        if self.add_accuracy:
            result.append(match.mean())
        if self.add_rating:
            result.append(rating.mean())
        return result


In [ ]:
def get_multilabel_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=dft.DataFrameTransformerFactory.default_factory())
                       )
    return label_extractor


In [ ]:
class ClassificationTask(TaskFactory):
    def create_task(self, data, env):
        metrics = bt.MetricPool().add(MulticlassMetrics())  # TODO epoch_count=20
        self.instantiate_default_task(
            epoch_count=1, batch_size=10000, mini_batch_size=None, metric_pool=metrics)
        self.setup_batcher(
            # data, [get_feature_extractor(), get_multilabel_extractor()])
            data, [core, get_multilabel_extractor()])
        self.setup_model(lambda _, sample: ClassificationNetwork(
            20, sample), learning_rate=1)


task = ClassificationTask()
idb = IndexedDataBundle(db.index, db)
result = task.run(idb)
# pd.DataFrame(result['output']['history']).set_index('iteration').plot()


setup_butcher принимает экстракторы. Передадим core? или завернем его в get_feature_extractor?